In [3]:
from shroomdk import ShroomDK

# Initialize `ShroomDK` with your API Key
sdk = ShroomDK("ce7ee339-c712-4e83-a06b-6039a058ca43")

In [12]:
from datetime import datetime
import pandas as pd

# DAILY QUERY RUN LIMIT 10000
# BURST QUERY RUN LIMIT 250 (maybe 5miniutes?)
DATE_FORMAT ='%Y-%m-%d %H:%M:%S'

dex_list = { 
    'ether' : {
        'uniswap_v3' : 'ethereum.uniswapv3.ez_swaps' 
    }
}
df_list = {}


def querying_pagination(query_string):
    result_list = []
    for i in range(1,11): # max is a million rows @ 100k per page
        data=sdk.query(query_string,page_size=100000,page_number=i)
        if data.run_stats.record_count == 0:  
            break
        else:
            result_list.append(data.records)
        
    # Loops through the returned results and adds into a pandas dataframe
    result_df=pd.DataFrame()
    for idx, each_list in enumerate(result_list):
        if idx == 0:
            result_df=pd.json_normalize(each_list)
        else:
            result_df=pd.concat([result_df, pd.json_normalize(each_list)])

    return result_df



def querying_swap(start_date,end_date,chain='ether',dex='uniswap_v3'):
    sql = f"""
        SELECT * 
        FROM {dex_list[chain][dex]}
        WHERE BLOCK_TIMESTAMP >= '{start_date}' AND BLOCK_TIMESTAMP <= '{end_date}'
        ORDER BY BLOCK_TIMESTAMP
    """
    df = querying_pagination(sql)
    if chain not in df_list:
        df_list[chain]={}
    if dex in df_list[chain]:
        df_list[chain][dex] = pd.concat([df_list[chain][dex], df])
        df_list[chain][dex].drop_duplicates(inplace=True)
        df_list[chain][dex].sort_values('block_timestamp')
        df_list[chain][dex].reset_index(drop=True,inplace=True)
    else:
        df_list[chain][dex]=df        
            
def aggregate_per_block(start_date,end_date,chain='ether',dex='uniswap_v3'):
    df = df_list[chain][dex]
    df['block_timestamp'] = pd.to_datetime(df['block_timestamp'])
    df = df[(df['block_timestamp']>=datetime.strptime(start_date,DATE_FORMAT))&(df['block_timestamp']<datetime.strptime(end_date,DATE_FORMAT))]
    df['ratio'] = df['amount0_adjusted']/df['amount1_adjusted']
    aggreagted_df = df.groupby(['block_number','block_timestamp','token0_symbol','token1_symbol','ratio']).count()[['blockchain']].rename(columns = {'blockchain' : 'count'})
    # aggreagted_df.reset_index(inplace=True)
    return aggreagted_df

In [13]:
querying_swap(start_date = '2023-04-03 18:00:11',end_date = '2023-04-03 19:00:11')

In [22]:
aggregate_per_block(start_date = '2023-04-03 18:30:11',end_date = '2023-04-03 19:00:11')

C:\Users\sangwon\AppData\Local\Temp\ipykernel_32140\2747243105.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ratio'] = df['amount0_adjusted']/df['amount1_adjusted']


count
block_number block_timestamp     token0_symbol token1_symbol ratio               
16970278     2023-04-03 18:30:11 USDC          USDT          -1.000422e+00      1
                                 WBTC          USDC          -3.555542e-05      1
                                               WETH          -6.447408e-02      1
                                 WETH          USDT          -5.515155e-04      1
                                 XEN           WETH          -6.229313e+08      1
...                                                                           ...
16970424     2023-04-03 18:59:47 USDC          WETH          -1.833105e+03      1
16970425     2023-04-03 18:59:59 DAI           WETH          -1.833245e+03      1
                                 DC            WETH          -1.029264e+06      1
                                 USDC          WETH          -1.832630e+03      1
                                 WETH          USDT          -5.456660e-04      1

[1466 rows x 1 columns]

In [68]:
# filterd_df = test_df[(test_df['block_number']==16970426) & (test_df['token0_symbol']=='WETH') & (test_df['token1_symbol']=='USDT')]
# filterd_df['ratio'] = filterd_df['amount0_adjusted']/filterd_df['amount1_adjusted']
# filterd_df[['amount0_adjusted','amount1_adjusted','ratio']]

C:\Users\sangwon\AppData\Local\Temp\ipykernel_36012\735773252.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filterd_df['ratio'] = filterd_df['amount0_adjusted']/filterd_df['amount1_adjusted']


,amount0_adjusted,amount1_adjusted,ratio
2472,-59.803287,109715.771593,-0.000545
2473,-3.409570,6258.481819,-0.000545
2474,-5.447523,10000.001291,-0.000545
